# **Importing Necessary Libraries**

In [2]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
plt.style.use('ggplot')
np.random.seed(1234)

np.set_printoptions(formatter={'all':lambda x: '%.3f' % x})

from IPython.display import Image
from numpy.core.umath_tests import matrix_multiply as mm

from scipy.optimize import minimize
from scipy.stats import bernoulli, binom

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  


# **Solving for Complete Likelihood**

In [0]:
def neg_loglik(thetas, n, xs, zs):
    return -np.sum([binom(n, thetas[z]).logpmf(x) for (x, z) in zip(xs, zs)])

In [5]:
# setting initial parameters for mean and standard deviation
m = 10
theta_A = 0.8
theta_B = 0.3
theta_0 = [theta_A, theta_B]

# conducting bernoulli trials for both coins
coin_A = bernoulli(theta_A)
coin_B = bernoulli(theta_B)

# create dataset for coin tosses
xs = map(sum, [coin_A.rvs(m), coin_A.rvs(m), coin_B.rvs(m), coin_A.rvs(m), coin_B.rvs(m)])

# here we have the complete information of which coins were tossed
zs = [0, 0, 1, 0, 1]

bnds = [(0,1), (0,1)]
minimize(neg_loglik, [0.5, 0.5], args=(m, xs, zs),
         bounds=bnds, method='tnc', options={'maxiter': 100})

     fun: -0.0
     jac: array([0.000, 0.000])
 message: 'Local minimum reached (|pg| ~= 0)'
    nfev: 1
     nit: 0
  status: 0
 success: True
       x: array([0.500, 0.500])

# **Extrapolating the solution to solve for an incomplete likelihood**

In [15]:
heads = [14, 33, 19, 10, 0, 17, 24, 17, 1, 36, 5, 6, 5, 13, 4, 35, 5, 5, 74, 34]
throws = [41, 43, 23, 23, 1, 23, 36, 37, 2, 131, 5, 29, 13, 47, 10, 58, 15, 14, 100, 113]

arr = []
for i in range(len(heads)):
    arr.append((heads[i], throws[i]))

xs = np.array(arr)
thetas = np.array([[0.6, 0.4], [0.5, 0.5]])

tol = 0.01
max_iter = 100

ll_old = 0
for i in range(max_iter):
    ws_A = []
    ws_B = []

    vs_A = []
    vs_B = []

    ll_new = 0

    # E-step: calculate probability distributions over possible completions
    for x in xs:

        # multinomial (binomial) log likelihood
        ll_A = np.sum([x*np.log(thetas[0])])
        ll_B = np.sum([x*np.log(thetas[1])])

        # [EQN 1]
        denom = np.exp(ll_A) + np.exp(ll_B)
        w_A = np.exp(ll_A)/denom
        w_B = np.exp(ll_B)/denom

        ws_A.append(w_A)
        ws_B.append(w_B)

        # used for calculating theta
        vs_A.append(np.dot(w_A, x))
        vs_B.append(np.dot(w_B, x))

        # update complete log likelihood
        ll_new += w_A * ll_A + w_B * ll_B

    # M-step: update values for parameters given current distribution
    # [EQN 2]
    thetas[0] = np.sum(vs_A, 0)/np.sum(vs_A)
    thetas[1] = np.sum(vs_B, 0)/np.sum(vs_B)
    # print distribution of z for each x and current parameter estimate

    print("Iteration: %d" % (i+1))
    print("theta_A = %.2f, theta_B = %.2f, ll = %.2f" % (thetas[0,0], thetas[1,0], ll_new))

    if np.abs(ll_new - ll_old) < tol:
        break
    ll_old = ll_new

Iteration: 1
theta_A = 0.38, theta_B = 0.32, ll = -779.13
Iteration: 2
theta_A = 0.39, theta_B = 0.26, ll = -694.84
Iteration: 3
theta_A = 0.40, theta_B = 0.24, ll = -684.80
Iteration: 4
theta_A = 0.40, theta_B = 0.24, ll = -683.44
Iteration: 5
theta_A = 0.40, theta_B = 0.24, ll = -683.37
Iteration: 6
theta_A = 0.40, theta_B = 0.24, ll = -683.37


In [0]:
# code adapted from https://people.duke.edu/~ccc14/sta-663/EMAlgorithm.html

def em(xs, thetas, max_iter=100, tol=1e-6):
    """Expectation-maximization for coin sample problem."""

    ll_old = -np.infty
    for i in range(max_iter):
        ll = np.array([np.sum(xs * np.log(theta), axis=1) for theta in thetas])
        lik = np.exp(ll)
        ws = lik/lik.sum(0)
        vs = np.array([w[:, None] * xs for w in ws])
        thetas = np.array([v.sum(0)/v.sum() for v in vs])
        ll_new = np.sum([w*l for w, l in zip(ws, ll)])
        if np.abs(ll_new - ll_old) < tol:
            break
        ll_old = ll_new
    return i, thetas, ll_new

In [20]:
xs = np.array(arr)
thetas = np.array([[0.6, 0.4], [0.5, 0.5]])

#run the EM function
i, thetas, ll = em(xs, thetas)
print(i)
for theta in thetas:
    print(theta)
print(ll)
np.random.seed(111)

10
[0.401 0.599]
[0.235 0.765]
-683.3671795030095


Here, we notice that we get the same results by using the EM algorithm as we did in the expanded method above.